In [1]:
import pandas as pd
import censusdata
import os
from tqdm import tqdm
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)
# os.chdir('/media/data/athena/census/')

In [2]:
v1 = list(censusdata.censustable('acs5', 2018, 'B01001').keys())
v2 = list(censusdata.censustable('acs5', 2018, 'B02001').keys())
v3 = ['B12001_001E', 'B12001_002E', 'B12001_003E', 'B12001_004E', 'B12001_009E', 'B12001_010E', 'B12001_011E', 'B12001_012E', 'B12001_013E', 'B12001_018E', 'B12001_019E']
v4 = list(censusdata.censustable('acs5', 2018, 'B16001').keys())
v5 = list(censusdata.censustable('acs5', 2018, 'C17002').keys())
v6 = ['B06010_001E', 'B06010_002E', 'B06010_003E', 'B06010_004E', 'B06010_005E', 'B06010_006E', 'B06010_007E', 'B06010_008E', 'B06010_009E', 'B06010_010E', 'B06010_011E', 'B19058_001E', 'B19058_002E', 'B19058_003E', 'B21001_001E', 'B21001_002E', 'B21001_003E']
v7 = list(censusdata.censustable('acs5', 2018, 'B23001').keys())
v8 = ['B23006_001E', 'B23006_002E', 'B23006_009E', 'B23006_023E']
variables = v1+v2+v3+v4+v5+v6+v7+v8

In [3]:
geo = censusdata.geographies(censusdata.censusgeo([('state', '*')]), 'acs5', 2018)
for i in ['Puerto Rico', 'Alaska', 'Hawaii',]:
    try:
        geo.pop(i)
    except:
        pass

In [ ]:
source = 'acs5'
year = 2018
dfs = []
for i, v in tqdm(geo.items()):
    geographies = censusdata.censusgeo([v.geo[0], ('county', '*'), ('tract', '*')])
    df = censusdata.download(source, year, geographies, variables)
    dfs.append(df)
acs = pd.concat(dfs)

 20%|████████████████▋                                                                 | 10/49 [02:02<09:23, 14.46s/it]

In [ ]:
acs.to_csv('data/acs.csv')

In [ ]:
acs = pd.read_csv('data/acs.csv', index_col=0)

In [ ]:
acs.index.to_series().apply(extract_gidtr).values

In [ ]:
import re

def extract_gidtr(text):
    state = re.findall('state:([0-9]{2}?)>', text)[0]
    county = re.findall('county:([0-9]{3}?)>', text)[0]
    tract = re.findall('tract:([0-9]{6}?)$', text)[0]
    gidtr = state+county+tract
    return int(gidtr)

In [ ]:
acs['GIDTR'] = acs.index.to_series().apply(extract_gidtr).values

In [ ]:
# Download census planning data "https://www2.census.gov/adrm/PDB/2018/docs/pdb2018trv4_us.zip"
cp = pd.read_csv('data/pdb2018trv4_us.csv', encoding = "ISO-8859-1", usecols=['GIDTR', 'State', 'State_name', 'County', 'County_name', 'Tract', 'LAND_AREA', 'Tot_Population_CEN_2010', 'Tot_Population_ACS_12_16'])

In [ ]:
data = pd.merge(acs, cp, on='GIDTR')

In [ ]:
data.to_csv('data/acs_planning.csv')